In [ ]:
!pip install datasets
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_name='t5-small'

tokenizer = AutoTokenizer.from_pretrained(model_name)

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to('cuda')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**LOAD DATASET**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle (1).json to kaggle (1).json


In [ ]:
import pandas as pd

df = pd.read_csv("qa.csv")
print(df.head())
print(df.columns)


                                            question  \
0  Apa nama negara yang terlibat dalam perjanjian...   
1  Siapa yang menandatangani perjanjian memorandu...   
2  Di mana lokasi penandatanganan perjanjian memo...   
3  Apakah perjanjian memorandum saling pengertian...   
4  Berapa lama masa berlaku perjanjian memorandum...   

                                     answer  \
0                                negara(X).   
1  penandatangan(indonesia, Nama, menteri).   
2           lokasi_penandatanganan(Tempat).   
3                   dapat_diperpanjang(ya).   
4                       masa_berlaku(Lama).   

                                             context  
0  % Fakta-fakta umum\r\nnegara(indonesia).\r\nne...  
1  % Fakta-fakta umum\r\nnegara(indonesia).\r\nne...  
2  % Fakta-fakta umum\r\nnegara(indonesia).\r\nne...  
3  % Fakta-fakta umum\r\nnegara(indonesia).\r\nne...  
4  % Fakta-fakta umum\r\nnegara(indonesia).\r\nne...  
Index(['question', 'answer', 'context'], dtype='ob

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load CSV
df = pd.read_csv("qa.csv")

# 2. Optional: Strip whitespace dari kolom
df["question"] = df["question"].str.strip()
df["context"] = df["context"].str.strip()
df["answer"] = df["answer"].str.strip()

# 3. Split data: 80% train, 10% validation, 10% test
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# 4. Convert ke HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "validation": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

# 5. Save ke disk (opsional, supaya bisa di-load lagi nanti)
dataset.save_to_disk("qa_dataset")

# 6. Print summary
print(dataset)


Saving the dataset (0/1 shards):   0%|          | 0/3795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/474 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/475 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context', '__index_level_0__'],
        num_rows: 3795
    })
    validation: Dataset({
        features: ['question', 'answer', 'context', '__index_level_0__'],
        num_rows: 474
    })
    test: Dataset({
        features: ['question', 'answer', 'context', '__index_level_0__'],
        num_rows: 475
    })
})


In [ ]:
from datasets import DatasetDict

# Misalnya dataset kamu bernama `dataset`
dataset = DatasetDict({
    'train': dataset['train'].remove_columns(['__index_level_0__']),
    'validation': dataset['validation'].remove_columns(['__index_level_0__']),
    'test': dataset['test'].remove_columns(['__index_level_0__']),
})


In [ ]:
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 3795
    })
    validation: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 474
    })
    test: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 475
    })
})


In [ ]:
dataset.save_to_disk("qa_dataset_cleaneddd")

Saving the dataset (0/1 shards):   0%|          | 0/3795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/474 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/475 [00:00<?, ? examples/s]

In [ ]:
dataset['test'][0]

{'question': 'Kapan dan di mana memorandum saling pengertian antara pemerintah republik indonesia dan pemerintah republik persatuan tanzania mengenai kerja sama di sektor energi  ditandatangani?',
 'answer': 'ditandatangani(Tempat, tanggal(Hari, Bulan, Tahun)).',
 'context': '% Fakta Umum\r\npihak(indonesia).\r\npihak(tanzania).\r\n\r\n% Fakta Penandatangan\r\nmenteri(indonesia, arifin_tasrif).\r\nmenteri(tanzania, january_y_makamba).\r\n\r\n% Lokasi dan Tanggal Penandatangan\r\nditandatangani(dar_es_salaam, tanggal(22, agustus, 2023)).\r\n\r\n% Bahasa dan Validitas\r\ndokumen_berbahasa(indonesia).\r\ndokumen_berbahasa(inggris).\r\nversi_berlaku(jika_perbedaan, inggris).\r\n\r\n% Tujuan\r\ntujuan_msp(kerjasama_di_bidang([perminyakan, ketenagalistrikan, energi_terbarukan])).\r\n\r\n% Ruang Lingkup\r\ntidak_mengikat(msp).\r\npengecualian_pasals([pasal7, pasal8]).\r\ndiperlukan_perjanjian_terpisah.\r\n\r\n% Bidang Kerja Sama\r\nbidang_kerjasama(eksplorasi_potensi_investasi_perminyakan).\r

**Preprocess the Datasets**

In [ ]:
def tokenize_function(example):

#     print(len(example["question"]))
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(example['context'], example['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
#     print(prompt[0])
#     print()

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("Loaded Tokenized Dataset")
except:
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer'])

    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("Tokenized and Saved Dataset")

Map:   0%|          | 0/3795 [00:00<?, ? examples/s]

Map:   0%|          | 0/474 [00:00<?, ? examples/s]

Map:   0%|          | 0/475 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3795 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/474 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/475 [00:00<?, ? examples/s]

Tokenized and Saved Dataset


In [ ]:
print(tokenized_datasets.keys())
print(tokenized_datasets['train'][0].keys())
print(tokenized_datasets['train'][0]['input_ids'][:10])
print(tokenized_datasets['train'][0]['labels'][:10])
print(tokenized_datasets)

dict_keys(['train', 'validation', 'test'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 13102, 599, 2566, 51, 18994, 1571, 162]
[13102, 599, 834, 6, 3, 834, 6, 3, 834, 6]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3795
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 474
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 475
    })
})


In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (3795, 2)
Validation: (474, 2)
Test: (475, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3795
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 474
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 475
    })
})


**Test the Model with Zero Shot Inferencing**

In [ ]:
index = 0

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (1214 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
% Fakta Umum
pihak(indonesia).
pihak(tanzania).

% Fakta Penandatangan
menteri(indonesia, arifin_tasrif).
menteri(tanzania, january_y_makamba).

% Lokasi dan Tanggal Penandatangan
ditandatangani(dar_es_salaam, tanggal(22, agustus, 2023)).

% Bahasa dan Validitas
dokumen_berbahasa(indonesia).
dokumen_berbahasa(inggris).
versi_berlaku(jika_perbedaan, inggris).

% Tujuan
tujuan_msp(kerjasama_di_bidang([perminyakan, ketenagalistrikan, energi_terbarukan])).

% Ruang Lingkup
tidak_mengikat(msp).
pengecualian_pasals([pasal7, pasal8]).
diperlukan_perjanjian_terpisah.

% Bidang Kerja Sama
bidang_kerjasama(eksplorasi_potensi_investasi_perminyakan).
bidang_kerjasama(pertukaran_data_perminyakan).
bidang_kerjasama(kolaborasi_pengembangan_ketenagalistrikan_energi_terbarukan).
bidang_kerjasama(promosi_investasi_ketenagalistrikan_energi_terbarukan).
bidang_kerjasama(berbagi_pengeta

**Perform Full Fine-Tuning**

In [ ]:
try:
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
    finetuned_model = finetuned_model.to('cuda')
    to_train = False

except:
    to_train = True
    finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
    finetuned_model = finetuned_model.to('cuda')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
%%time

if to_train:
    output_dir = f'./sql-training-{str(int(time.time()))}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=2,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        eval_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=500,                     # number of steps between evaluation
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
    )

    trainer.train()

    finetuned_model.save_pretrained("finetuned_model_2_epoch")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hutapeatarutung123 (hutapeatarutung123-universitas-gunadarma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


CPU times: user 9min 25s, sys: 3min 37s, total: 13min 2s
Wall time: 14min 4s


In [ ]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_2_epoch")
finetuned_model = finetuned_model.to('cuda')

**Test the Fine Tuned Model with Zero Shot Inferencing**

In [ ]:
index = 0
# index = len(dataset['test'])-200

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cuda')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')


Token indices sequence length is longer than the specified maximum sequence length for this model (1214 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
% Fakta Umum
pihak(indonesia).
pihak(tanzania).

% Fakta Penandatangan
menteri(indonesia, arifin_tasrif).
menteri(tanzania, january_y_makamba).

% Lokasi dan Tanggal Penandatangan
ditandatangani(dar_es_salaam, tanggal(22, agustus, 2023)).

% Bahasa dan Validitas
dokumen_berbahasa(indonesia).
dokumen_berbahasa(inggris).
versi_berlaku(jika_perbedaan, inggris).

% Tujuan
tujuan_msp(kerjasama_di_bidang([perminyakan, ketenagalistrikan, energi_terbarukan])).

% Ruang Lingkup
tidak_mengikat(msp).
pengecualian_pasals([pasal7, pasal8]).
diperlukan_perjanjian_terpisah.

% Bidang Kerja Sama
bidang_kerjasama(eksplorasi_potensi_investasi_perminyakan).
bidang_kerjasama(pertukaran_data_perminyakan).
bidang_kerjasama(kolaborasi_pengembangan_ketenagalistrikan_energi_terbarukan).
bidang_kerjasama(promosi_investasi_ketenagalistrikan_energi_terbarukan).
bidang_kerjasama(berbagi_pengeta

**Evaluate the Model Quantitatively (with ROUGE Metric)**

In [ ]:
# Perform inferences for test dataset. Do 25 only, due to time it takes.

questions = dataset['test'][0:25]['question']
contexts = dataset['test'][0:25]['context']
human_baseline_answers = dataset['test'][0:25]['answer']

original_model_answers = []
finetuned_model_answers = []

for idx, question in enumerate(questions):

    prompt = f"""Tables:
{contexts[idx]}

Question:
{question}

Answer:
"""

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    input_ids = input_ids.to('cuda')

    human_baseline_text_output = human_baseline_answers[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_answers.append(original_model_text_output)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300))
    finetuned_model_text_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_text_output)

zipped_summaries = list(zip(human_baseline_answers, original_model_answers, finetuned_model_answers))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_answers', 'original_model_answers', 'finetuned_model_answers'])
# df

`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0, 'eos_token_id': 1, 'decoder_start_token_id': 0}. If this is not desired, please set these values explicitly.


In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=df679ba03dc4454d449b6dd3926ba65d9ca7a35a2232be66b2eac25793661e5b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_answers,
    references=human_baseline_answers[0:len(original_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)


finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=human_baseline_answers[0:len(finetuned_model_answers)],
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.09868399180109672), 'rouge2': np.float64(0.04905139964889395), 'rougeL': np.float64(0.09317833331392561), 'rougeLsum': np.float64(0.09444294763698394)}
FINE-TUNED MODEL:
{'rouge1': np.float64(0.11243707982838416), 'rouge2': np.float64(0.08), 'rougeL': np.float64(0.11587378804770108), 'rougeLsum': np.float64(0.11780615519745954)}


In [ ]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import string, re

# ====================
# FUNGSI NORMALISASI TEKS
# ====================
def normalize_text(s):
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return ''.join(ch for ch in text if ch not in set(string.punctuation))
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

# ====================
# METRIK MANUAL
# ====================
def compute_exact_match(pred, truth):
    return int(normalize_text(pred) == normalize_text(truth))

def compute_f1(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    common = set(pred_tokens) & set(truth_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    return 2 * (precision * recall) / (precision + recall)

def compute_bleu(pred, truth):
    pred_tokens = pred.lower().split()
    truth_tokens = truth.lower().split()
    smoothie = SmoothingFunction().method4
    return sentence_bleu([truth_tokens], pred_tokens, smoothing_function=smoothie)

def compute_execution_match(pred, truth):
    try:
        return int(eval(pred) == eval(truth))
    except:
        return 0

# ====================
# CONTOH JAWABAN
# ====================
finetuned_model_answers = ["3 + 5", "the capital is paris", "no", "1 / 0", "yes"]
human_baseline_answers = ["8", "paris is the capital", "no", "error", "yes"]

# ====================
# EVALUASI
# ====================
em_scores = []
f1_scores = []
bleu_scores = []
execution_scores = []
true_labels = [1] * len(human_baseline_answers)
predicted_labels = []

for pred, truth in zip(finetuned_model_answers, human_baseline_answers):
    em = compute_exact_match(pred, truth)
    f1 = compute_f1(pred, truth)
    bleu = compute_bleu(pred, truth)
    exec_match = compute_execution_match(pred, truth)

    em_scores.append(em)
    f1_scores.append(f1)
    bleu_scores.append(bleu)
    execution_scores.append(exec_match)
    predicted_labels.append(em)  # anggap EM sebagai label biner 1/0

# ====================
# HASIL AKHIR
# ====================
accuracy = accuracy_score(true_labels, predicted_labels)
f1_binary = f1_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)

print("==== EVALUASI MODEL ====")
print(f"Accuracy: {accuracy:.2%}")
print(f"F1 Score: {f1_binary:.2%}")
print(f"Recall: {recall:.2%}")
print(f"Exact Match: {sum(em_scores)/len(em_scores):.2%}")
print(f"Average F1 (Token-Based): {sum(f1_scores)/len(f1_scores):.2%}")
print(f"Average BLEU Score: {sum(bleu_scores)/len(bleu_scores):.2%}")
print(f"Execution Match Score: {sum(execution_scores)/len(execution_scores):.2%}")


==== EVALUASI MODEL ====
Accuracy: 40.00%
F1 Score: 57.14%
Recall: 40.00%
Exact Match: 40.00%
Average F1 (Token-Based): 60.00%
Average BLEU Score: 44.00%
Execution Match Score: 20.00%


In [ ]:
!pip install pyswip


In [ ]:
!apt-get install swi-prolog


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-sendmail-perl libossp-uuid16 libsub-override-perl
  libsys-hostname-long-perl libtool po-debconf swi-prolog-core
  swi-prolog-core-packages swi-prolog-doc swi-prolog-nox swi-prolog-x
Suggested packages:
  dh-make gettext-doc libasprintf-dev libgettextpo-dev uuid libtool-doc
  gcj-jdk libmail-box-perl elpa-ediprolog swi-prolog-java swi-prolog-odbc
  swi-prolog-bdb
The following NEW packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-send

In [ ]:
from pyswip import Prolog

# Fungsi Exact Match (EM)
def exact_match(predictions, references):
    """
    Menghitung skor Exact Match (EM) antara prediksi dan referensi.
    """
    em_score = sum([1 if pred == ref else 0 for pred, ref in zip(predictions, references)])
    return em_score / len(references)

# Fungsi Execution Match (XM) dengan PySWIP (Prolog)
def execution_match(predictions, references):
    """
    Menghitung Execution Match (XM) antara prediksi dan referensi menggunakan Prolog.
    """
    prolog = Prolog()
    match_score = 0

    for pred, ref in zip(predictions, references):
        prolog.assertz(f"{pred}")  # Menambahkan klausa ke Prolog
        result = list(prolog.query(ref))  # Mengeksekusi klausa
        if result:
            match_score += 1
        prolog.retractall(f"{pred}")  # Menghapus klausa setelah eksekusi

    return match_score / len(references)

# Contoh penggunaan:
predictions = ["penandatangan(indonesia, menteri_luar_negeri)"]  # Pastikan tidak ada titik ganda
references = ["penandatangan(indonesia, menteri_luar_negeri)"]

# Menghitung metrik EM dan XM
em_score = exact_match(predictions, references)
xm_score = execution_match(predictions, references)

print(f"Exact Match (EM) Score: {em_score}")
print(f"Execution Match (XM) Score: {xm_score}")


Exact Match (EM) Score: 1.0
Execution Match (XM) Score: 1.0
